In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Loan Prediction Dataset**

While browsing through the net for suitable project for a beginner like me, I found this practice problem in Analytics Vidya website. 

It is a classification problem wherein we have to predict whether loan will be approved or not based on the features given.

I now load important libraries.
The data was given in seperate train and test folders were the target variable was present only in the train folder.
Hence, we cannot evaluate the classification model instead need to upload the solution file( .csv) in their website which will give the accuracy score of the model.

In [ ]:
# Loan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import shapiro
import seaborn as sns

# Dataset
a = pd.read_csv(r'/kaggle/input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv')
del a['Loan_ID']
a.Dependents = a.Dependents.replace({"3+": "3"})
X_train = a.iloc[:,:-1]
Y_train = a.iloc[:,-1]
b = pd.read_csv(r'/kaggle/input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv')
c = b['Loan_ID']
del b['Loan_ID']
b.Dependents = b.Dependents.replace({"3+": "3"})
X_test = b.iloc[:,:]

names = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Gender_Female','Gender_Male',
         'Married_No','Married_Yes','Dependents_0','Dependents_1','Dependents_2','Dependents_3','Education_Graduate',
         'Education_Not Graduate','Self_Employed_No','Self_Employed_Yes','Property_Area_Rural','Property_Area_Semiurban', 'Property_Area_Urban']


I deleted Loan_Id for analysis but saved it under different array because it has to be included in submission file.

In [ ]:
# EDA
X_train.isnull().sum()


In [ ]:
X_test.isnull().sum()

Since, null values are present,
I will impute them ( for continuous features ) and for categorical data, I will replace them by mode of the column and then encode it using getdummies.

I did not include Pregnancies for imputation because this coulmn can have 0.
So, the only continuous data that has 0 values are Loan amount and Loan amount Term which I will impute.


In [ ]:

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train.iloc[:,[7,8]])
X_train.iloc[:, [ 7, 8]] = imputer.fit_transform(X_train.iloc[:, [7,8]])
X_test.iloc[:, [ 7, 8]] = imputer.fit_transform(X_test.iloc[:, [7,8]])



Gender, Married, Education, Self - Employed, Property and Credit_History are categorical. These will be encoded.


In [ ]:
# Gender, Married, Education, Self - Employed, Property and Credit_History are categorical.
for i in ['Gender', 'Married', 'Dependents', 'Self_Employed','Credit_History']:
    X_train[i].fillna(X_train[i].mode()[0], inplace=True)
    X_test[i].fillna(X_test[i].mode()[0], inplace=True)



In [ ]:

# Encoding categorical data
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
Y_train = lb.fit_transform(Y_train)


In [ ]:
# Distributions
fig, axes = plt.subplots(nrows=2, ncols=2, figsize = (12,12))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('Distributions of Features')
for ax, feature in zip(axes.flatten(),  names[0:4]):
    sns.distplot(X_train[feature],  ax=ax)
plt.show()


The distributions of continuous features are slightly right skewed. I will use log on them in feature engineering.

In [ ]:
# Checking for correlations
fig, ax = plt.subplots(figsize=(15,15))         
sns.heatmap(X_train.corr().values.reshape(20,20), annot = True, annot_kws={'fontsize':6}, ax = ax)
plt.plot()
plt.show()
# All correlations are between the encoded variables
# Otherwise no correlation


In [ ]:
# Model
# Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)

# Prediction
Y_pred = classifier.predict(X_test)
ans = lb.inverse_transform(Y_pred)
solution = pd.DataFrame(list(zip(c, ans)), columns = ['Loan_ID', "Loan_Status"])


I built a simple logistic regression model and stored the predictions in a .csv file for submission.

In [ ]:
# Feature Engineering
# Outlier Detection
fig, ax = plt.subplots(figsize=(10,10))         
sns.boxplot(data = X_train[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']],palette = 'Set2', orient = "h", ax = ax)
plt.show()


There are many outliers, I will cap them.

In [ ]:
outlier1 = []
def outlier(df):
    for i in (names[0:3]):
        iqr = df[i].quantile(0.75) - df[i].quantile(0.25)
        up = df[i].quantile(0.75) + (iqr * 1.5)
        low = df[i].quantile(0.25) - (iqr * 1.5)
        print(i,up, low)
        for j in range(614):
            if df[i][j] < low:
                df.replace(df[i][j], low, inplace = True)
                outlier1.append(df[i][j])
            elif df[i][j] > up:
                df.replace(df[i][j], up, inplace = True)
                outlier1.append(df[i][j])


outlier(X_train)
print(f"There are {len(outlier1)} outliers in the dataset. I have capped these values.")


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))         
sns.boxplot(data = X_train[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']],palette = 'Set2', orient = "h", ax = ax)
plt.show()


Outliers have been capped.

Now I will do log + 1 transformation and check the distributions.

In [ ]:

# Log transformation
X_train[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']] = \
    np.log(X_train[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']] +1)

# Distributions
fig, axes = plt.subplots(nrows=2, ncols=2, figsize = (12,12))
fig.subplots_adjust(hspace=0.5)
fig.suptitle('Distributions of Features')
for ax, feature in zip(axes.flatten(),  names[0:4]):
    sns.distplot(X_train[feature],  ax=ax)
plt.show()



After running the model and saving the predictions using solution.to_csv(r'E:\Notes\Projects\Submission_Loan.csv'),
I got accracy of 0.7152.


![1.png](http:///kaggle/input/images/1.png)